In [1]:
"""
Train convolutional network for sentiment analysis. Based on
"Convolutional Neural Networks for Sentence Classification" by Yoon Kim
http://arxiv.org/pdf/1408.5882v2.pdf

For 'CNN-non-static' gets to 82.1% after 61 epochs with following settings:
embedding_dim = 20          
filter_sizes = (3, 4)
num_filters = 3
dropout_prob = (0.7, 0.8)
hidden_dims = 100

For 'CNN-rand' gets to 78-79% after 7-8 epochs with following settings:
embedding_dim = 20          
filter_sizes = (3, 4)
num_filters = 150
dropout_prob = (0.25, 0.5)
hidden_dims = 150

For 'CNN-static' gets to 75.4% after 7 epochs with following settings:
embedding_dim = 100          
filter_sizes = (3, 4)
num_filters = 150
dropout_prob = (0.25, 0.5)
hidden_dims = 150

* it turns out that such a small data set as "Movie reviews with one
sentence per review"  (Pang and Lee, 2005) requires much smaller network
than the one introduced in the original article:
- embedding dimension is only 20 (instead of 300; 'CNN-static' still requires ~100)
- 2 filter sizes (instead of 3)
- higher dropout probabilities and
- 3 filters per filter size is enough for 'CNN-non-static' (instead of 100)
- embedding initialization does not require prebuilt Google Word2Vec data.
Training Word2Vec on the same "Movie reviews" data set is enough to 
achieve performance reported in the article (81.6%)

** Another distinct difference is sliding MaxPooling window of length=2
instead of MaxPooling over whole feature map as in the article
"""

'\nTrain convolutional network for sentiment analysis. Based on\n"Convolutional Neural Networks for Sentence Classification" by Yoon Kim\nhttp://arxiv.org/pdf/1408.5882v2.pdf\n\nFor \'CNN-non-static\' gets to 82.1% after 61 epochs with following settings:\nembedding_dim = 20          \nfilter_sizes = (3, 4)\nnum_filters = 3\ndropout_prob = (0.7, 0.8)\nhidden_dims = 100\n\nFor \'CNN-rand\' gets to 78-79% after 7-8 epochs with following settings:\nembedding_dim = 20          \nfilter_sizes = (3, 4)\nnum_filters = 150\ndropout_prob = (0.25, 0.5)\nhidden_dims = 150\n\nFor \'CNN-static\' gets to 75.4% after 7 epochs with following settings:\nembedding_dim = 100          \nfilter_sizes = (3, 4)\nnum_filters = 150\ndropout_prob = (0.25, 0.5)\nhidden_dims = 150\n\n* it turns out that such a small data set as "Movie reviews with one\nsentence per review"  (Pang and Lee, 2005) requires much smaller network\nthan the one introduced in the original article:\n- embedding dimension is only 20 (inste

In [2]:
import numpy as np
import data_helpers
from w2v import train_word2vec

from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Input, Merge, Convolution1D, MaxPooling1D

from sklearn.cross_validation import train_test_split

np.random.seed(2)

Using Theano backend.


In [3]:
model_variation = 'CNN-rand'  #  CNN-rand | CNN-non-static | CNN-static
print('Model variation is %s' % model_variation)

Model variation is CNN-rand


In [4]:
# Model Hyperparameters
sequence_length = 56
embedding_dim = 20          
filter_sizes = (3, 4)
num_filters = 150
dropout_prob = (0.25, 0.5)
hidden_dims = 150

In [5]:
# Training parameters
batch_size = 32
num_epochs = 2

In [6]:
# Word2Vec parameters, see train_word2vec
min_word_count = 1  # Minimum word count                        
context = 10        # Context window size    

In [7]:
print("Loading data...")
x, y, vocabulary, vocabulary_inv = data_helpers.load_data()

if model_variation=='CNN-non-static' or model_variation=='CNN-static':
    embedding_weights = train_word2vec(x, vocabulary_inv, embedding_dim, min_word_count, context)
    if model_variation=='CNN-static':
        x = embedding_weights[0][x]
elif model_variation=='CNN-rand':
    embedding_weights = None
else:
    raise ValueError('Unknown model variation')    

Loading data...


In [8]:
data = np.append(x,y,axis = 1)

In [9]:
train, test = train_test_split(data, test_size = 0.15,random_state = 0)

In [10]:
X_test = test[:,:56]
Y_test = test[:,56:58]


X_train = train[:,:56]
Y_train = train[:,56:58]
train_rows = np.random.randint(0,X_train.shape[0],2500)
X_train = X_train[train_rows]
Y_train = Y_train[train_rows]

In [11]:
print("Vocabulary Size: {:d}".format(len(vocabulary)))


Vocabulary Size: 18779


In [12]:
def initialize():
    
    global graph_in
    global convs
    
    graph_in = Input(shape=(sequence_length, embedding_dim))
    convs = []

In [13]:
#Buliding the first layer (Convolution Layer) of the network
def build_layer_1(filter_length):
    
   
    conv = Convolution1D(nb_filter=num_filters,
                         filter_length=filter_length,
                         border_mode='valid',
                         activation='relu',
                         subsample_length=1)(graph_in)
    return conv

In [14]:
#Adding a max pooling layer to the model(network)
def add_max_pooling(conv):
    
    pool = MaxPooling1D(pool_length=2)(conv)
    return pool

In [15]:
#Adding a flattening layer to the model(network), before adding a dense layer
def add_flatten(conv_or_pool):
    
    flatten = Flatten()(conv_or_pool)
    return flatten

In [16]:
def add_sequential(graph):
    
    #main sequential model
    model = Sequential()
    if not model_variation=='CNN-static':
        model.add(Embedding(len(vocabulary), embedding_dim, input_length=sequence_length,
                        weights=embedding_weights))
    model.add(Dropout(dropout_prob[0], input_shape=(sequence_length, embedding_dim)))
    model.add(graph)
    model.add(Dense(2))
    model.add(Activation('sigmoid'))
    
    return model

In [17]:
#1.Convolution 2.Flatten
def one_layer_convolution():
    
    initialize()
    
    conv = build_layer_1(3)
    flatten = add_flatten(conv)
    
    convs.append(flatten)
    out = convs[0]

    graph = Model(input=graph_in, output=out)
    
    model = add_sequential(graph)
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    model.fit(X_train, Y_train, batch_size=batch_size,
          nb_epoch=num_epochs, validation_data=(X_test, Y_test))
    
    score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [18]:
#1.Convolution 2.Max Pooling 3.Flatten
def two_layer_convolution():
    
    initialize()
    
    conv = build_layer_1(3)
    pool = add_max_pooling(conv)
    flatten = add_flatten(pool)
    
    convs.append(flatten)
    out = convs[0]

    graph = Model(input=graph_in, output=out)
    
    model = add_sequential(graph)
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    model.fit(X_train, Y_train, batch_size=batch_size,
          nb_epoch=num_epochs, validation_data=(X_test, Y_test))
    
    score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [19]:
#1.Convolution 2.Max Pooling 3.Flatten 4.Convolution 5.Flatten
def three_layer_convolution():
    
    initialize()
    
    conv = build_layer_1(3)
    pool = add_max_pooling(conv)
    flatten = add_flatten(pool)
    
    convs.append(flatten)
    
    conv = build_layer_1(4)
    flatten = add_flatten(conv)
    
    convs.append(flatten)
    
    if len(filter_sizes)>1:
        out = Merge(mode='concat')(convs)
    else:
        out = convs[0]

    graph = Model(input=graph_in, output=out)
    
    model = add_sequential(graph)
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    model.fit(X_train, Y_train, batch_size=batch_size,
          nb_epoch=10, validation_data=(X_test, Y_test))
    
    score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [20]:
#1.Convolution 2.Max Pooling 3.Flatten 4.Convolution 5.Max Pooling 6.Flatten
def four_layer_convolution():
    
    initialize()
    
    conv = build_layer_1(3)
    pool = add_max_pooling(conv)
    flatten = add_flatten(pool)
    
    convs.append(flatten)
    
    conv = build_layer_1(4)
    pool = add_max_pooling(conv)
    flatten = add_flatten(pool)
    
    convs.append(flatten)
    
    if len(filter_sizes)>1:
        out = Merge(mode='concat')(convs)
    else:
        out = convs[0]

    graph = Model(input=graph_in, output=out)
    
    model = add_sequential(graph)
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    model.fit(X_train, Y_train, batch_size=batch_size,
          nb_epoch=num_epochs, validation_data=(X_test, Y_test))
    
    score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [21]:
%%time
#1.Convolution 2.Flatten
one_layer_convolution()

Train on 2500 samples, validate on 1600 samples
Epoch 1/2
2500/2500 [==============================] - 26s - loss: 0.6936 - acc: 0.4992 - val_loss: 0.6938 - val_acc: 0.4769
Epoch 2/2
2500/2500 [==============================] - 26s - loss: 0.6930 - acc: 0.5152 - val_loss: 0.6926 - val_acc: 0.5231
Test score: 0.692598519325
Test accuracy: 0.523125
CPU times: user 1min 6s, sys: 54.4 s, total: 2min
Wall time: 59 s


In [21]:
%%time
#1.Convolution 2.Max Pooling 3.Flatten
two_layer_convolution()

Train on 2500 samples, validate on 1600 samples
Epoch 1/2
2500/2500 [==============================] - 21s - loss: 0.6934 - acc: 0.4920 - val_loss: 0.6931 - val_acc: 0.5081
Epoch 2/2
2500/2500 [==============================] - 21s - loss: 0.6930 - acc: 0.5072 - val_loss: 0.6923 - val_acc: 0.5231
Test score: 0.692291852236
Test accuracy: 0.523125
CPU times: user 47.5 s, sys: 396 ms, total: 47.9 s
Wall time: 48.1 s


In [21]:
%%time
#1.Convolution 2.Max Pooling 3.Flatten 4.Convolution 5.Flatten
three_layer_convolution()

Train on 2500 samples, validate on 1600 samples
Epoch 1/10
2500/2500 [==============================] - 53s - loss: 0.6937 - acc: 0.5004 - val_loss: 0.6941 - val_acc: 0.4769
Epoch 2/10
2500/2500 [==============================] - 50s - loss: 0.6929 - acc: 0.5132 - val_loss: 0.6928 - val_acc: 0.5344
Epoch 3/10
2500/2500 [==============================] - 53s - loss: 0.6920 - acc: 0.5340 - val_loss: 0.6951 - val_acc: 0.4769
Epoch 4/10
2500/2500 [==============================] - 52s - loss: 0.6899 - acc: 0.5504 - val_loss: 0.6918 - val_acc: 0.5031
Epoch 5/10
2500/2500 [==============================] - 58s - loss: 0.6803 - acc: 0.6160 - val_loss: 0.6800 - val_acc: 0.6056
Epoch 6/10
2500/2500 [==============================] - 57s - loss: 0.6298 - acc: 0.6988 - val_loss: 0.6862 - val_acc: 0.5775
Epoch 7/10
2500/2500 [==============================] - 53s - loss: 0.5074 - acc: 0.7752 - val_loss: 0.6048 - val_acc: 0.6713
Epoch 8/10
2500/2500 [==============================] - 56s - loss: 0.

In [21]:
%%time
#1.Convolution 2.Max Pooling 3.Flatten 4.Convolution 5.Max Pooling 6.Flatten
four_layer_convolution()

Train on 2500 samples, validate on 1600 samples
Epoch 1/2
2500/2500 [==============================] - 50s - loss: 0.6937 - acc: 0.4976 - val_loss: 0.6927 - val_acc: 0.5231
Epoch 2/2
2500/2500 [==============================] - 51s - loss: 0.6929 - acc: 0.5148 - val_loss: 0.6926 - val_acc: 0.5231
Test score: 0.692577679157
Test accuracy: 0.523125
CPU times: user 1min 58s, sys: 54.6 s, total: 2min 53s
Wall time: 1min 52s
